<a href="https://colab.research.google.com/github/scbadkas19/NLP/blob/master/countvectorizer%20in%20pipline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import io
import regex as re
import nltk
from zipfile import ZipFile
from urllib.request import urlopen
import pandas as pd
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import classification_report,accuracy_score,confusion_matrix
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

In [0]:
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip'

In [5]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
stoplist = set(stopwords.words('english'))

In [8]:
resp = urlopen(url)
zipfile = ZipFile(io.BytesIO(resp.read()))
zipfile.namelist()

['SMSSpamCollection', 'readme']

In [0]:
smspam = pd.read_csv(zipfile.open('SMSSpamCollection'),sep = '\t',header=None)

In [10]:
smspam

,0,1
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [0]:
def prepipe(text):
  text=re.sub('[^a-zA-Z]', " ", str(text))
  text=text.lower()
  data = ' '.join([w for w in word_tokenize(text) if w not in stoplist])
  return data

In [0]:
smspam['processd'] = smspam[1].apply(prepipe)

In [14]:
smspam['processd']

0       go jurong point crazy available bugis n great ...
1                                 ok lar joking wif u oni
2       free entry wkly comp win fa cup final tkts st ...
3                     u dun say early hor u c already say
4                  nah think goes usf lives around though
                              ...                        
5567    nd time tried contact u u pound prize claim ea...
5568                            b going esplanade fr home
5569                                pity mood suggestions
5570    guy bitching acted like interested buying some...
5571                                       rofl true name
Name: processd, Length: 5572, dtype: object

In [0]:
steps = [('text',CountVectorizer()),('ensemble',RandomForestClassifier())]
pipeline = Pipeline(steps) 
params = {'text__stop_words':['english'],'ensemble__max_depth': [10, 20],'ensemble__min_samples_leaf': [1, 2],'ensemble__min_samples_split': [2, 5],'ensemble__n_estimators': range(10,50,20)}

In [0]:
grid = GridSearchCV(pipeline, param_grid=params, cv=5,verbose=1)

In [0]:
x_train,x_test,y_train,y_test = tts(smspam['processd'],smspam[0])

In [18]:
grid.fit(x_train,y_train)

Fitting 5 folds for each of 16 candidates, totalling 80 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  80 out of  80 | elapsed:    8.8s finished


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('text',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                                        prep

In [19]:
grid.score(x_test,y_test)

0.9454414931801867